This notebook is intended for Python 2 with Spark 2.0. It uses SparkSession to load a CSV file stored in Bluemix object storage into a dataframe, filters that data, then writes the filtered data to a previoulsy created Cloudant database. This example notebook loads a CSV file containing Child Care providers in Massachusetts downloaded from https://data.mass.gov/Education/Program-list-for-Child-Care-Search-1-15-2015/cb6m-ccic

This first cell simply verifies the version of Spark you are using.

In [1]:
spark.version

u'2.0.2'

Cell 2: Replace the contents of the first cell by following these steps:
1. Displaying the Files slide out panel.
2. Select the Insert to code menu for your file, and select Insert Credentials.
3. Replace the name of the inserted array with credentials_621 as referenced in the rest of the code. 

In [2]:
# The code was removed by DSX for sharing.

Cell 3: The following cell imports SparkSession from pyspark.sql. SparkSession is the entry point to programming Spark with the Dataset and DataFrame API.
Next, the code defines a variable to set the credentials for authentication for the Bluemix Object Storage.

In [3]:
# The code was removed by DSX for sharing.

Cell 4: The following cell reads the CSV file into a data frame, infers the schema, and then displays the first two entries.

In [4]:
massdata = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('timestampFormat', 'MM/dd/yyyy')\
  .option('inferSchema', 'true')\
  .load('swift://' + credentials_621['container'] + '.' + name + '/' + credentials_621['filename'])
massdata.take(2)

[Row(ProgramName=u'Little Prince and Princess Day School', ProgramType=u'Large Group and School Age Child Care', FirstIssueDate=datetime.datetime(1983, 9, 26, 0, 0), Capacity=43, Licensor=u'Collyer, Renee - 5G011', DirectorFirstName=u'Ruthie', DirectorLastName=u'Harris', Address1=u'877 N Main St', Address2=None, City=u'Brockton', State=u'MA', Zip=u'02301-1623', ProviderPhone=u'5085872483', FaxNumber=None, MailingAddress1=None, MailingAddress2=None, MailingCity=None, MailingState=u'  ', MailingZipcode=None, eMailAddress=u'ruthieharris2@verizon.net', Latitude=42.106159, Longitude=-71.026859, Transportation=u'Transportation Provided; Walking Distance to School; Near Public Transportation; On Public School Bus Route', Meals=u'Breakfast; Morning Snack; Afternoon Snack; Special Meal Request; Parents Provide Food; Parents Prov', FinancialAssistance=None, SpecialNeeds=u'Experience; ADD/ADHD; Asthma/Allergies; Behavioral; Developmental; Emotional/Social; Learning; Medi', Session1Name=u'Session 

Cell 5: The following cell prints the schema and a record count of the data frame contents.

In [5]:
massdata.printSchema()
massdata.count()

root
 |-- ProgramName: string (nullable = true)
 |-- ProgramType: string (nullable = true)
 |-- FirstIssueDate: timestamp (nullable = true)
 |-- Capacity: integer (nullable = true)
 |-- Licensor: string (nullable = true)
 |-- DirectorFirstName: string (nullable = true)
 |-- DirectorLastName: string (nullable = true)
 |-- Address1: string (nullable = true)
 |-- Address2: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zip: string (nullable = true)
 |-- ProviderPhone: string (nullable = true)
 |-- FaxNumber: long (nullable = true)
 |-- MailingAddress1: string (nullable = true)
 |-- MailingAddress2: string (nullable = true)
 |-- MailingCity: string (nullable = true)
 |-- MailingState: string (nullable = true)
 |-- MailingZipcode: string (nullable = true)
 |-- eMailAddress: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Transportation: string (nullable = true)
 |-- Meals: 

8282

Cell 6: The following cell displays the first 30 values in the Session1Name field. Notice that there are null values.

In [6]:
massdata.select("Session1Name").show(30)

+------------+
|Session1Name|
+------------+
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 3|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 3|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|        null|
|   Session 3|
|   Session 1|
|   Session 1|
|        null|
|   Session 1|
|   Session 1|
+------------+
only showing top 30 rows



Cell 7: The following cell filters the data to just those facilities that have a specified Session1Name. Then it displays the first two entries and a count of the filtered data.

In [7]:
sessiondata = massdata.filter(massdata.Session1Name.isNotNull())
sessiondata.show(2)
sessiondata.count()

+--------------------+--------------------+--------------------+--------+--------------------+-----------------+----------------+--------------+--------+--------+-----+----------+-------------+----------+---------------+---------------+-----------+------------+--------------+--------------------+---------+----------+--------------------+--------------------+-------------------+--------------------+------------+------------------------+----------------+---------------+------------------+----------------+---------------------+------------------+--------------------+------------+------------------------+----------------+---------------+------------------+----------------+---------------------+------------------+--------------------+------------+------------------------+----------------+---------------+------------------+----------------+---------------------+------------------+--------------------+
|         ProgramName|         ProgramType|      FirstIssueDate|Capacity|            Licens

6780

Cell 8: The following cell displays the first 30 values in the Session1Name field. Notice that there are NO null values.

In [8]:
sessiondata.select("Session1Name").show(30)

+------------+
|Session1Name|
+------------+
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 3|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 3|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 3|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 1|
|   Session 2|
|   Session 1|
+------------+
only showing top 30 rows



Cell 9: The following cell writes the contents of the sessiondata data frame to a Cloudant database called child_care. Note: The Cloudant database MUST already exist.

In [9]:
sessiondata.write.format("com.cloudant.spark") \
  .option("cloudant.host","345b1802-97ce-40d8-8e4a-bca356c4eadf-bluemix.cloudant.com") \
  .option("cloudant.username","345b1802-97ce-40d8-8e4a-bca356c4eadf-bluemix") \
  .option("cloudant.password","d86064e90676b18bcd88d49c0e7070eac4d3f36b0a50d1c9c5ee4919583b24ce") \
  .save("child_care")